In [ ]:
from pathlib import Path
import mne
import hcp
import hcp.preprocessing as preproc
mne.set_log_level('WARNING')

In [ ]:
def extract_activation(
                        subject,
                        hcp_path,
                        subjects_dir=None,
                        freq_band=(0.5, 60),
                        source=True,
                        annot="aparc",
                        snr=1.0,
                        saving_dir=None,
                        create_report=True,
                        ):
    
    '''
    
    '''

In [ ]:
## 1. dl the MEG anatomy folder
## 2. dl the structural extended preprocess
## 3. dl the processed MEG 
## 4. dl the unprocessed MEG

## create folder structure
## 1. create trans folder if its not there and make it writable
## 2. check if I have MRI folder
## 3. per subject move T1w folder from struc ext preprocess to MEG/anatomy folder 
## 4. run hcp.make_mne_anatomy to create subjects_dir

## checks

## establish everything

## now the real part

subject = '100307'
run_index = 1
data_type = 'rest'
hcp_path = Path('/Volumes/Extreme_SSD/payam_data/hcp')
hcp_path_proc = hcp_path / 'processed'
hcp_path_unproc = hcp_path / 'unprocessed'
subjects_dir = hcp_path / 'subjects'


l_freq, h_freq = 0.1, 80

kwargs_unproc = {
                "subject": subject,
                "run_index": 1,
                "data_type": "rest",
                "hcp_path": hcp_path / "unprocessed"
                }

kwargs_proc = {
                "subject": subject,
                "run_index": 1,
                "data_type": "rest",
                "hcp_path": hcp_path / "processed"
                }

# create subjects_dir
try:
    hcp.make_mne_anatomy(subject=subject,
                    subjects_dir=hcp_path / 'subjects',
                    hcp_path=hcp_path / 'mri',
                    recordings_path=hcp_path / 'trans'
                    )
except:
    pass

## read necessary information
info = hcp.read_info(**kwargs_unproc)
annots = hcp.read_annot(**kwargs_proc)
ica = hcp.read_ica(**kwargs_proc)
raw = hcp.read_raw(**kwargs_unproc)
raw.load_data()



## preprocess
preproc.apply_ref_correction(raw)

bad_seg = (annots['segments']['all']) / raw.info['sfreq']
annotations = mne.Annotations(
                                bad_seg[:, 0], (bad_seg[:, 1] - bad_seg[:, 0]),
                                description='bad'
                                )
raw.set_annotations(annotations)
raw.info['bads'].extend(annots['channels']['all'])
raw.pick_types(meg=True, ref_meg=False)

## filter
raw.filter(l_freq, h_freq, method='iir', iir_params=dict(order=4, ftype='butter'), n_jobs=1, verbose=False)

## ica
exclude = [ii for ii in range(annots['ica']['total_ic_number'][0]) if ii not in annots['ica']['brain_ic_vs']]
preproc.apply_ica_hcp(raw, ica_mat=ica, exclude=exclude)

## handling empty room noise

## setup source space
src = mne.setup_source_space(subject=subject,
                                subjects_dir=hcp_path / 'subjects',
                                add_dist=True,
                                spacing='oct6'
                                )

head_mri_t = mne.read_trans(fname=hcp_path / 'trans' / f"{subject}" /f"{subject}-head_mri-trans.fif")
bems = mne.make_bem_model(subject,
                            conductivity=(0.3,),
                            subjects_dir=hcp_path / 'subjects',
                            ico=None
                            )
bem_sol = mne.make_bem_solution(bems)
bem_sol['surfs'][0]['coord_frame'] = 5

picks = mne.pick_types(raw.info, meg=True, ref_meg=False)
info = mne.pick_info(raw.info, picks)

fwd = mne.make_forward_solution(raw.info,
                                src=src,
                                trans=head_mri_t,
                                bem=bem_sol
                                )


## estimate noise covariance
raw_noise = hcp.read_raw(subject=subject,
                        hcp_path=hcp_path,
                        data_type='noise_empty_room'
                        )

raw_noise.filter(l_freq, h_freq, method='iir', iir_params=dict(order=4, ftype='butter'), n_jobs=1)
noise_cov = mne.compute_raw_covariance(raw_noise, method='empirical')
del raw_noise

inv_op = mne.minimum_norm.make_inverse_operator(raw.info,
                                                fwd,
                                                noise_cov
                                                )
stcs = mne.minimum_norm.apply_inverse_raw(raw,
                                    inv_op,
                                    lambda2=1.0 / snr**2
                                    )

## extract signals from parcels
annot_fname = ""
parc = 'aparc'
extraction_mode = "mean"
labels = mne.read_labels_from_annot(subject,
                                    parc=parc,
                                    annot_fname=annot_fname
                                    subjects_dir=subjects_dir
                                    )
label_ts = mne.extract_label_time_course(stcs,
                                        labels,
                                        src,
                                        allow_empty=True,
                                        mode=extraction_mode,
                                        return_generator=True
                                        )

## save the matrix 
